<a href="https://colab.research.google.com/github/MrTyoom/Project-Jackal/blob/ml-experiments/Jackal_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импорт датасета

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r'first_command_new.csv')
df2 = pd.read_csv(r'commands-w2v-version.csv')

In [ ]:
df[:3]
df2[:3]

Подготовка и реализация word2vec

In [ ]:
import gensim
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec

In [ ]:
def generate_dictinoary_data(text):
    word_to_index= dict()
    index_to_word = dict()
    corpus = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in row.split():
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index,index_to_word,corpus,vocab_size,length_of_corpus


In [ ]:
generate_dictinoary_data(df['command'])

In [ ]:
# get the text data from the CSV file
text_data = df['command'].tolist()

# preprocess the text data
preprocessed_text_data = []
for text in text_data:
    text = text.lower()
    text = nltk.word_tokenize(text)
    preprocessed_text_data.append(text)

# train a Word2Vec model on the preprocessed text data
model = Word2Vec(preprocessed_text_data, vector_size = 100, window=5, min_count=1, workers=4)

In [ ]:
# use the Word2Vec model to get word embeddings
embedding = model.wv['двигать']

In [ ]:
embedding

Получили словарь слов из всех команд и вектор для каждого слова в словаре

Теперь сделаем эмбеддинги с помощью другой модели  — SBERT

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer as SBERT
sbert1 = SBERT('sberbank-ai/sbert_large_nlu_ru')


In [ ]:
EMBEDDING = sbert1.encode([s for s in df.command])
EMBEDDING2 = sbert1.encode([s for s in df2.command])

Получили эмбеддинги из SBERT'a

  \

Получим модель из эмбеддингов с помощью Сatboost'a

In [ ]:
!pip install catboost

In [ ]:
import catboost
from catboost import * 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['lables'] = le.fit_transform(df.intent)
df2['lables'] = le.fit_transform(df2.intent)

In [ ]:
X = df.drop("lables", axis=1)
y = df["lables"]

X2 = df2.drop("lables", axis=1)
y2 = df2["lables"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(EMBEDDING, y, test_size=0.33, random_state=42)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(EMBEDDING2, y2, test_size=0.33, random_state=42)

In [ ]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(iterations=100,
                         learning_rate=0.2,
                         depth=2, custom_loss=['AUC', 'Accuracy'])

model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
          verbose=50, plot=True
)

print(f"Model is fitted: {str(model.is_fitted())}")
print(f"Model params: {model.get_params()}")


In [ ]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(iterations=100,
                         learning_rate=0.2,
                         depth=2, custom_loss=['AUC', 'Accuracy'])

model.fit(X_train2, y_train2,
         eval_set=(X_test2, y_test2),
          verbose=50, plot=True
)

print(f"Model is fitted: {str(model.is_fitted())}")
print(f"Model params: {model.get_params()}")


Еще модели для сравнения

In [ ]:
model1 = CatBoostClassifier(
    learning_rate=0.7,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.7'
)

model2 = CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.01'
)

model1.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=50
)

model2.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=50
)

In [ ]:
model3 = CatBoostClassifier(
    learning_rate=0.7,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.7'
)

model4 = CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    random_seed=63,
    train_dir='learning_rate_0.01'
)

model3.fit(
    X_train2, y_train2,
    eval_set=(X_test2, y_test2),
    verbose=50
)

model4.fit(
    X_train2, y_train2,
    eval_set=(X_test2, y_test2),
    verbose=50
)

Теперь делаем predict

In [ ]:
y_pred = model2.predict(X_test, 
        prediction_type='Class', 
        ntree_start=0, 
        ntree_end=0, 
        thread_count=-1,
        verbose=None)

In [ ]:
y_pred2 = model4.predict(X_test2, 
        prediction_type='Class', 
        ntree_start=0, 
        ntree_end=0, 
        thread_count=-1,
        verbose=None)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred) 

Заметим, что классы: 0 — корабль влево, 1 — корабль вправо, 2 — ничего. Тогда пом матрице видно, что класс "ничего" никогда не определяется как лево/право (3я строчка матрицы), право было интерпретировано как ничего 2 раза, лево 1 (3й столбец матрицы.)

При этом, ошибочное определение "право" как лево было в 20 случаях, наоборот в 50. 

Делаем вывод, что классы не различаются между собой.

Здесь я вывожу график классов, уменьшив размерность с помощью T-SNE

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test2, y_pred2) 

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE